In [1]:
# test_connection.py
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

# .env 파일 로드
load_dotenv()

# 환경 변수 읽기
URI = os.getenv("NEO4J_URI")
USERNAME = os.getenv("NEO4J_USERNAME")
PASSWORD = os.getenv("NEO4J_PASSWORD")
DATABASE = os.getenv("NEO4J_DATABASE")

print(f"연결 시도 중...")
print(f"URI: {URI}")
print(f"Username: {USERNAME}")
print(f"Database: {DATABASE}")

# Neo4j 연결
try:
    driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))
    
    # 연결 테스트
    with driver.session(database=DATABASE) as session:
        result = session.run("RETURN 'Hello, Neo4j!' AS message")
        record = result.single()
        print(f"\n✅ 연결 성공!")
        print(f"메시지: {record['message']}")
    
    driver.close()
    
except Exception as e:
    print(f"\n❌ 연결 실패!")
    print(f"에러: {e}")

연결 시도 중...
URI: None
Username: None
Database: None

❌ 연결 실패!
에러: URI scheme b'' is not supported. Supported URI schemes are ['bolt', 'bolt+ssc', 'bolt+s', 'neo4j', 'neo4j+ssc', 'neo4j+s']. Examples: bolt://host[:port] or neo4j://host[:port][?routing_context]


In [ ]:
# setup_schema.py
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

def setup_schema():
    """제약조건과 인덱스 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        
        # 제약조건
        constraints = [
            "CREATE CONSTRAINT user_id IF NOT EXISTS FOR (u:User) REQUIRE u.userId IS UNIQUE",
            "CREATE CONSTRAINT product_id IF NOT EXISTS FOR (p:Product) REQUIRE p.productId IS UNIQUE",
            "CREATE CONSTRAINT category_name IF NOT EXISTS FOR (c:Category) REQUIRE c.name IS UNIQUE"
        ]
        
        # 인덱스
        indexes = [
            "CREATE INDEX user_name IF NOT EXISTS FOR (u:User) ON (u.name)",
            "CREATE INDEX user_email IF NOT EXISTS FOR (u:User) ON (u.email)",
            "CREATE INDEX product_name IF NOT EXISTS FOR (p:Product) ON (p.name)"
        ]
        
        print("📋 스키마 생성 중...")
        
        # 제약조건 생성
        for constraint in constraints:
            try:
                session.run(constraint)
                print(f"✅ {constraint[:50]}...")
            except Exception as e:
                print(f"⚠️  이미 존재: {constraint[:50]}...")
        
        # 인덱스 생성
        for index in indexes:
            try:
                session.run(index)
                print(f"✅ {index[:50]}...")
            except Exception as e:
                print(f"⚠️  이미 존재: {index[:50]}...")
        
        print("\n✅ 스키마 생성 완료!")
        
setup_schema()
driver.close()

In [ ]:
# testing cell
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)
with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
    query = "MATCH (n) RETURN count(n) AS node_count"
    result = session.run(query, forceerror=["ex1", "ex2"])
    print(result.single()["node_count"])
driver.close()

In [ ]:
# generate_data.py
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker('ko_KR')  # 한국어 더미 데이터

driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

# 카테고리 목록
CATEGORIES = ['전자제품', '의류', '식품', '가구', '도서', '스포츠', '뷰티', '완구']

def clear_database():
    """기존 데이터 삭제"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print("🗑️  기존 데이터 삭제 중...")
        session.run("MATCH (n) DETACH DELETE n")
        print("✅ 삭제 완료!")

def create_categories():
    """카테고리 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print(f"\n📂 카테고리 {len(CATEGORIES)}개 생성 중...")
        
        query = """
        UNWIND $categories AS category
        MERGE (c:Category {name: category})
        RETURN count(c) AS count
        """
        
        result = session.run(query, categories=CATEGORIES)
        count = result.single()['count']
        print(f"✅ {count}개 카테고리 생성 완료!")

def create_users(num_users=100):
    """사용자 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print(f"\n👥 사용자 {num_users}명 생성 중...")
        
        users = []
        for i in range(1, num_users + 1):
            users.append({
                'userId': i,
                'name': fake.name(),
                'email': fake.email(),
                'phone': fake.phone_number(),
                'address': fake.address(),
                'joinedAt': fake.date_time_between(start_date='-2y', end_date='now').isoformat()
            })
        
        query = """
        UNWIND $users AS user
        CREATE (u:User {
            userId: user.userId,
            name: user.name,
            email: user.email,
            phone: user.phone,
            address: user.address,
            joinedAt: datetime(user.joinedAt)
        })
        """
        
        session.run(query, users=users)
        print(f"✅ {num_users}명 사용자 생성 완료!")

def create_products(num_products=200):
    """상품 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print(f"\n🛍️  상품 {num_products}개 생성 중...")
        
        products = []
        for i in range(1, num_products + 1):
            products.append({
                'productId': i,
                'name': fake.catch_phrase(),
                'description': fake.text(max_nb_chars=100),
                'price': round(random.uniform(5000, 500000), -2),  # 5천원~50만원
                'stock': random.randint(0, 100),
                'category': random.choice(CATEGORIES)
            })
        
        query = """
        UNWIND $products AS product
        CREATE (p:Product {
            productId: product.productId,
            name: product.name,
            description: product.description,
            price: product.price,
            stock: product.stock
        })
        WITH p, product
        MATCH (c:Category {name: product.category})
        CREATE (p)-[:IN_CATEGORY]->(c)
        """
        
        session.run(query, products=products)
        print(f"✅ {num_products}개 상품 생성 완료!")

def create_purchases(num_purchases=500):
    """구매 관계 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print(f"\n💰 구매 내역 {num_purchases}개 생성 중...")
        
        # 사용자와 상품 ID 범위 가져오기
        user_count = session.run("MATCH (u:User) RETURN count(u) AS count").single()['count']
        product_count = session.run("MATCH (p:Product) RETURN count(p) AS count").single()['count']
        
        purchases = []
        for _ in range(num_purchases):
            purchases.append({
                'userId': random.randint(1, user_count),
                'productId': random.randint(1, product_count),
                'quantity': random.randint(1, 5),
                'totalPrice': round(random.uniform(10000, 1000000), -2),
                'purchasedAt': fake.date_time_between(start_date='-1y', end_date='now').isoformat()
            })
        
        query = """
        UNWIND $purchases AS purchase
        MATCH (u:User {userId: purchase.userId})
        MATCH (p:Product {productId: purchase.productId})
        MERGE (u)-[r:PURCHASED]->(p)
        ON CREATE SET 
            r.quantity = purchase.quantity,
            r.totalPrice = purchase.totalPrice,
            r.purchasedAt = datetime(purchase.purchasedAt)
        """
        
        session.run(query, purchases=purchases)
        print(f"✅ {num_purchases}개 구매 내역 생성 완료!")

def create_reviews(num_reviews=300):
    """리뷰 생성"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print(f"\n⭐ 리뷰 {num_reviews}개 생성 중...")
        
        # 구매 관계 가져오기
        result = session.run("""
            MATCH (u:User)-[:PURCHASED]->(p:Product)
            RETURN u.userId AS userId, p.productId AS productId
            LIMIT $limit
        """, limit=num_reviews)
        
        reviews = []
        for record in result:
            reviews.append({
                'userId': record['userId'],
                'productId': record['productId'],
                'rating': random.randint(1, 5),
                'comment': fake.text(max_nb_chars=200),
                'reviewedAt': fake.date_time_between(start_date='-6m', end_date='now').isoformat()
            })
        
        query = """
        UNWIND $reviews AS review
        MATCH (u:User {userId: review.userId})
        MATCH (p:Product {productId: review.productId})
        MERGE (u)-[r:REVIEWED]->(p)
        SET r.rating = review.rating,
            r.comment = review.comment,
            r.reviewedAt = datetime(review.reviewedAt)
        """
        
        session.run(query, reviews=reviews)
        print(f"✅ {len(reviews)}개 리뷰 생성 완료!")

def print_statistics():
    """데이터 통계 출력"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        print("\n" + "="*50)
        print("📊 데이터베이스 통계")
        print("="*50)
        
        # 노드 수
        stats = [
            ("User", "MATCH (u:User) RETURN count(u) AS count"),
            ("Product", "MATCH (p:Product) RETURN count(p) AS count"),
            ("Category", "MATCH (c:Category) RETURN count(c) AS count")
        ]
        
        for label, query in stats:
            count = session.run(query).single()['count']
            print(f"  {label}: {count:,}개")
        
        print()
        
        # 관계 수
        rels = [
            ("PURCHASED", "MATCH ()-[r:PURCHASED]->() RETURN count(r) AS count"),
            ("REVIEWED", "MATCH ()-[r:REVIEWED]->() RETURN count(r) AS count"),
            ("IN_CATEGORY", "MATCH ()-[r:IN_CATEGORY]->() RETURN count(r) AS count")
        ]
        
        for rel_type, query in rels:
            count = session.run(query).single()['count']
            print(f"  {rel_type}: {count:,}개")
        
        print("="*50 + "\n")

# 1. 기존 데이터 삭제
clear_database()

# 2. 데이터 생성
create_categories()
create_users(num_users=100)
create_products(num_products=200)
create_purchases(num_purchases=500)
create_reviews(num_reviews=300)

# 3. 통계 출력
print_statistics()

print("✅ 모든 작업 완료!")
driver.close()

In [ ]:
# query_examples.py
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

def run_query(query, parameters=None):
    """쿼리 실행 및 결과 출력"""
    with driver.session(database=os.getenv("NEO4J_DATABASE")) as session:
        result = session.run(query, parameters or {})
        return [record.data() for record in result]

# 예시 쿼리들
queries = {
    "1. 전체 사용자 수": """
        MATCH (u:User) 
        RETURN count(u) AS userCount
    """,
    
    "2. 상위 10개 인기 상품": """
        MATCH (p:Product)<-[r:PURCHASED]-()
        RETURN p.name AS product, 
               count(r) AS purchaseCount,
               avg(r.totalPrice) AS avgPrice
        ORDER BY purchaseCount DESC
        LIMIT 10
    """,
    
    "3. 카테고리별 상품 수": """
        MATCH (p:Product)-[:IN_CATEGORY]->(c:Category)
        RETURN c.name AS category, 
               count(p) AS productCount
        ORDER BY productCount DESC
    """,
    
    "4. 특정 사용자의 구매 이력": """
        MATCH (u:User {userId: 1})-[r:PURCHASED]->(p:Product)
        RETURN u.name AS user,
               p.name AS product,
               r.quantity AS quantity,
               r.totalPrice AS price,
               r.purchasedAt AS date
        ORDER BY date DESC
    """,
    
    "5. 평점 높은 상품 추천": """
        MATCH (p:Product)<-[r:REVIEWED]-()
        WITH p, avg(r.rating) AS avgRating, count(r) AS reviewCount
        WHERE reviewCount >= 3
        RETURN p.name AS product,
               avgRating,
               reviewCount
        ORDER BY avgRating DESC, reviewCount DESC
        LIMIT 10
    """
}

if __name__ == "__main__":
    print("\n" + "="*60)
    print("📊 샘플 쿼리 실행")
    print("="*60 + "\n")
    
    for title, query in queries.items():
        print(f"{title}")
        print("-" * 60)
        results = run_query(query)
        
        if results:
            # 결과 출력
            for i, record in enumerate(results, 1):
                print(f"  {i}. {record}")
        else:
            print("  결과 없음")
        
        print()
    
    driver.close()